In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns
import xgboost as xgb




In [2]:
train_set = pd.read_csv('adult.data', header=None)
test_set = pd.read_csv('adult.test', skiprows=1, header=None) #do not forget to skip the first row 

In [3]:
train_set.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
test_set.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


**I notice a few problems immediately:**

1. We don’t have a column header for our data.


2. There seem to be some unknown values in the fifth row of the test set (the question marks) we need to deal with.


3. The target values have periods at the end in the test set but do not in the training set (<=50K. vs. <=50K).


Based on the accompanying dataset description, we can see the column names. Let’s put those in for our train and test first.

In [5]:
col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']

In [6]:
train_set.columns = col_labels
test_set.columns = col_labels

In [7]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
wage_class        32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [8]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 0 to 16280
Data columns (total 15 columns):
age               16281 non-null int64
workclass         16281 non-null object
fnlwgt            16281 non-null int64
education         16281 non-null object
education_num     16281 non-null int64
marital_status    16281 non-null object
occupation        16281 non-null object
relationship      16281 non-null object
race              16281 non-null object
sex               16281 non-null object
capital_gain      16281 non-null int64
capital_loss      16281 non-null int64
hours_per_week    16281 non-null int64
native_country    16281 non-null object
wage_class        16281 non-null object
dtypes: int64(6), object(9)
memory usage: 1.9+ MB


Next, let’s check to see if pandas has identified any of these missing values.

Haaa!, There don’t seem to be any. According to the accompanying notes(see the adult.names file), the original datasets had 32561 in train and 16281 with test. However, unknowns are included and have been labeled with a question mark (?). The test results documented were done after removing all unknowns. Therefore, to see if we can beat their best results, we need to remove the same unknown rows.

If we do so, we should have 30162 in train and 15060 in test. Let’s see if we can remove all of these unknown rows.

It turns out the question mark was actually entered with a space first. Let’s do a simple test to see what would happen if we dropped all rows that contain an unknown marked with a `‘ ?’`.



In [9]:
train_set.replace(' ?', np.nan).dropna().shape

(30162, 15)

In [10]:
test_set.replace(" ?", np.nan).dropna().shape

(15060, 15)

These must be our missing rows since the numbers add up now if we drop them. Let’s apply this change to our test and training sets.

In [11]:
train_nomissing = train_set.replace(' ?', np.nan).dropna()
test_nomissing = test_set.replace(' ?', np.nan).dropna()

Now that we have taken care of the missing value problem, we still have an issue with the target income thresholds being encoded slightly differently in test vs. train. We need these to match up appropriately, so we are going to need to fix either the test or training set to make them match up. Let’s replace all of the `‘<=50K.’` with `‘<=50K’` and the same for `‘>50K.’` with `‘>50K’`, so essentially, we are just dropping the periods. This is also encoded with a space so include this in the string. We can use the replace method from pandas to fix this.

In [12]:
test_nomissing['wage_class'] = test_nomissing.wage_class.replace(
    {' <=50K.': ' <=50K', ' >50K.':' >50K'})

In [13]:
train_nomissing['wage_class'] = train_nomissing.wage_class.replace(
    {' <=50K.': ' <=50K', ' >50K.':' >50K'})

Checking the unique values for `wage_class`, we can see they match now in the both classes.

In [14]:
test_nomissing.wage_class.unique()

array([' <=50K', ' >50K'], dtype=object)

In [15]:
train_nomissing.wage_class.unique()

array([' <=50K', ' >50K'], dtype=object)

There is one thing we need to do, however, before applying XGBoost. We need to make sure all categorical variables encoded as a string is turned into a numerical variable, as **xgboost does not accept categorical varibles**.

**The question is how to convert categorical features (especially nominal features) to numerical features**

Let's discuss some of the popular methods including their pros and cons and use fullness of each method in a separate notebook. Namely : **Concepts_of_TS(mean)_Encoding**



### Mean Encoding of Categorical Features:

First, combine them together into a single dataset.

In [16]:
combined_set = pd.concat([train_nomissing, test_nomissing], axis = 0) # Stacks them vertically

In [17]:
combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 16280
Data columns (total 15 columns):
age               45222 non-null int64
workclass         45222 non-null object
fnlwgt            45222 non-null int64
education         45222 non-null object
education_num     45222 non-null int64
marital_status    45222 non-null object
occupation        45222 non-null object
relationship      45222 non-null object
race              45222 non-null object
sex               45222 non-null object
capital_gain      45222 non-null int64
capital_loss      45222 non-null int64
hours_per_week    45222 non-null int64
native_country    45222 non-null object
wage_class        45222 non-null object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


In [18]:
wage_class_binary = pd.get_dummies(combined_set['wage_class'])

In [19]:
combined_set['wage_class']= wage_class_binary

In [20]:
means = combined_set.groupby('workclass')['wage_class'].mean()

In [21]:
means


workclass
 Federal-gov         0.609531
 Local-gov           0.704839
 Private             0.782298
 Self-emp-inc        0.445930
 Self-emp-not-inc    0.721022
 State-gov           0.732785
 Without-pay         0.904762
Name: wage_class, dtype: float64

In [22]:
cat_colums = ['workclass','education','marital_status','occupation','relationship',
                 'race','sex','native_country',]
              

In [23]:
for cat_feature in cat_colums:
    combined_set[cat_feature] = combined_set[cat_feature].map(combined_set.groupby(cat_feature)['wage_class'].mean())

In [24]:
combined_set.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,0.732785,77516,0.580185,13,0.951980,0.863538,0.895061,0.737629,0.687523,2174,0,40,0.746973,1
1,50,0.721022,83311,0.580185,13,0.545761,0.520889,0.544252,0.737629,0.687523,0,0,13,0.746973,1
2,38,0.782298,215646,0.836569,9,0.895982,0.934018,0.895061,0.737629,0.687523,0,0,40,0.746973,1
3,53,0.782298,234721,0.945028,7,0.545761,0.934018,0.544252,0.873699,0.687523,0,0,40,0.746973,1
4,28,0.782298,338409,0.580185,13,0.545761,0.549933,0.514108,0.873699,0.886424,0,0,40,0.744361,1


In [25]:
combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 16280
Data columns (total 15 columns):
age               45222 non-null int64
workclass         45222 non-null float64
fnlwgt            45222 non-null int64
education         45222 non-null float64
education_num     45222 non-null int64
marital_status    45222 non-null float64
occupation        45222 non-null float64
relationship      45222 non-null float64
race              45222 non-null float64
sex               45222 non-null float64
capital_gain      45222 non-null int64
capital_loss      45222 non-null int64
hours_per_week    45222 non-null int64
native_country    45222 non-null float64
wage_class        45222 non-null uint8
dtypes: float64(8), int64(6), uint8(1)
memory usage: 5.2 MB


Now that we have all of our features encoded, we need to split these back into their original train/test sizes. Since they haven’t been shuffled we just need to retrieve the same indices as before.

In [26]:
final_train = combined_set[:train_nomissing.shape[0]] # Up to the last initial training set row

final_test = combined_set[train_nomissing.shape[0]:] # Past the last initial training set row

We can now finally start playing around with XGBoost.

### Initial Model Setup and Grid Search

We still have our target value inside our train and test frames that needs to be separated from the feature vectors we will be feeding into XGBoost. Let’s get those now.

In [27]:
y_train = final_train.pop('wage_class')
y_test = final_test.pop('wage_class')

Now import the libraries we will need to do grid search for XGBoost. Fortunately for those of us used to sklearn’s API, XGBoost is compatible with this, so we can still utilize the traditional GridSearch with XGBoost.

In [28]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

To see all of the available parameters that can be tuned in XGBoost, have a look at the [parameter documentation](https://xgboost.readthedocs.io/en/latest/parameter.html). This should help you better understand the choices I am making to start off our first grid search. I am going to start tuning on the maximum depth of the trees first, along with the min_child_weight, which is very similar to min_samples_split in sklearn’s version of gradient boosted trees. We set the objective to ‘binary:logistic’ since this is a binary classification problem

In [29]:
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1,return_train_score=True) 
# Optimize for accuracy since that is the metric used in the Adult Data Set notation

Now let’s run our grid search with 5-fold cross-validation and see which parameters perform the best.

In [30]:
optimized_GBM.fit(final_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.8),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

Let’s check our grid scores.

In [31]:
optimized_GBM.cv_results_

{'mean_fit_time': array([19.08266101, 18.00571833, 17.43481426, 29.51810613, 31.96300235,
        29.43433008, 43.8946898 , 42.42584314, 38.4221602 ]),
 'std_fit_time': array([0.12033969, 0.85500965, 0.48750676, 2.07876085, 1.29696573,
        0.08233173, 1.51797837, 1.43439729, 5.49277112]),
 'mean_score_time': array([0.28540201, 0.29020205, 0.28760147, 0.59891748, 0.64901962,
        0.58771911, 1.28009634, 0.86642365, 0.77261386]),
 'std_score_time': array([0.00714409, 0.0101869 , 0.01100136, 0.06540441, 0.0989848 ,
        0.11401031, 0.40294839, 0.0825699 , 0.23528658]),
 'param_max_depth': masked_array(data=[3, 3, 3, 5, 5, 5, 7, 7, 7],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_min_child_weight': masked_array(data=[1, 3, 5, 1, 3, 5, 1, 3, 5],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_val

In [32]:
optimized_GBM.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.8)

In [33]:
optimized_GBM.best_score_

0.868410582852596

In [34]:
optimized_GBM.best_params_

{'max_depth': 3, 'min_child_weight': 1}

We can see that the first hyperparameter combination performed best and we already beat our target of 85.95% accuracy(see the adult.names file) in our cross-validation! please try optimizing some other hyperparameters now to see if we can beat a mean of 86.84% accuracy.

Hint: play around with subsampling along with lowering the learning rate to see if that helps.

In [ ]:
cv_params = {'learning_rate': [0.1, 0.01], 'subsample': [0.7,0.8,0.9]}

ind_params = {'n_estimators': 1000, 'seed':0, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth': 3, 'min_child_weight': 1}


optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1,return_train_score=True)

optimized_GBM.fit(final_train, y_train)

Again, check the grid scores.

In [ ]:
optimized_GBM.cv_results_

In [ ]:
optimized_GBM.best_score_

No, it doesn’t look like we can improve on this. However, we may be able to optimize a little further by utilizing XGBoost’s built-in cv which allows early stopping to prevent overfitting.

### Early Stopping

Based on the CV testing performed earlier, we want to utilize the following parameters:

- Learning_rate (eta) = 0.1


- Subsample, colsample_bytree = 0.8


- Max_depth = 3


- Min_child_weight = 1

There are a few other parameters we could tune in theory to squeeze out further performance, but this is a good enough starting point.

To **increase the performance of XGBoost’s speed** through many iterations of the training set, and since from now on we are using only **Python's Native XGBoost API** and not **sklearn’s xgboost API** anymore(skleran does not have early stopping facility), we can create a DMatrix. This sorts the data initially to optimize for XGBoost when it builds trees, making the algorithm more efficient. This is especially helpful when you have a very large number of training examples. To create a DMatrix:

In [ ]:
xgbdmat = xgb.DMatrix(final_train, y_train) # Create our DMatrix to make XGBoost more efficient

Now let’s specify our parameters (with slightly different syntax in some places for the **Pythons native XGBoost  API**) and set our early stopping criteria. For now, let’s be aggressive with the stopping and say we don’t want the accuracy to improve for at least 100 new trees.

In [ ]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth':3, 'min_child_weight':1} 
# Grid Search CV optimized settings

cv_xgb = xgb.cv(params = our_params, dtrain = xgbdmat, num_boost_round = 3000, nfold = 5,
                metrics = ['error'], # Make sure you enter metrics inside a list or you may encounter issues!
                early_stopping_rounds = 100) # Look for early stopping that minimizes error

We can look at our CV results to see how accurate we were with these settings. The output is automatically saved into a pandas dataframe for us.

In [ ]:
cv_xgb.tail(5)

Our CV test error at this number of iterations is 11.6388%, or 88.3612% accuracy.

Now that we have our best settings, let’s create this as an xgboost object model that we can reference later.

In [ ]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth':3, 'min_child_weight':1} 

final_xgb_model = xgb.train(our_params, xgbdmat, num_boost_round = 432)

In [ ]:
final_xgb_model.save_model('final_xgb_model0001.model')

The model and its feature map can also be dumped to a text file.

In [ ]:
# dump model
final_xgb_model.dump_model('final_xgb_model001.raw.txt')


### Feature Importance:

With our XGB model object, we can then plot our feature importances using a built-in method. This is similar to the feature importances found in sklearn.

In [ ]:
xgb.plot_importance(final_xgb_model)

This will tell us which features were most important in the series of trees. The `**fnlwgt**` feature seems to have the most importance. Filing `capital_gains` was also important, which makes sense given that only those with greater incomes have the ability to invest. `race` and `sex` were not as important.

### Analyzing Performance on Test Data


The model has now been tuned using cross-validation grid search through the sklearn API and early stopping through the built-in XGBoost API. Now, we can see how it finally performs on the test set. Does it match our CV performance? First, create another DMatrix (this time for the test data).

In [ ]:
testdmat = xgb.DMatrix(final_test)

In [ ]:
y_pred_prob = final_xgb_model.predict(testdmat) # Predict using our testdmat
y_pred_prob

You can see that the predict function for XGBoost outputs probabilities by default and not actual class labels. To calculate accuracy we need to convert these to a 0/1 label. We will set 0.5 probability as our threshold.

In [ ]:
y_pred[y_pred_prob > 0.5] = 1
y_pred[y_pred_prob <= 0.5] = 0
y_pred

In [ ]:
from sklearn.metrics import accuracy_score

Now we can calculate our accuracy.

In [ ]:
accuracy_score(y_pred, y_test), 1-accuracy_score(y_pred, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)


In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

###  Summary
In this post, we explored some of the basic functionality involving the XGBoost library. We also learned how it works and why it performs faster than other gradient boosting libraries do. As a test, we used it on an example dataset of US incomes, beating the performance of other documented models for the dataset with very little effort. We were also able to investigate feature importances to see which features were influencing the model most.We also measured the performance of our XGB Binary Classier using various metrics avaliable in sklearn.